# Notebook to explore the Gather data sets provided

General information:
 - there are multiple files containing preprocessed data

In [1]:
pwd

'/Users/hramsden/MyFiles/Projects/Gather/Gather Data Dive March 18/03_wsup_data/python'

In [2]:
data_dir = '../data/wsup/tidy/'

In [3]:
ls $data_dir

questionnaire_codebook.csv             tidy_data_questionnaire_section_5.csv
tidy_data_questionnaire_section_2.csv  tidy_data_questionnaire_section_7.csv
tidy_data_questionnaire_section_3.csv  tidy_data_questionnaire_section_8.csv
tidy_data_questionnaire_section_4.csv


In [4]:
import pandas_profiling
import pandas as pd

## Overview of the data files

In [16]:
data_overview = pd.read_csv(data_dir + 'questionnaire_codebook.csv', index_col = 0)
data_overview

,section,question,answer,variable_text,file
Questionnaire.question,,,,,
2.2 Description of respondent (landlord),2,2,28,respondent_info,tidy_data_section_2.csv
2.2 Description of respondent (caretaker),2,2,31,respondent_info,tidy_data_section_2.csv
2.2 Description of respondent (tenant),2,2,34,respondent_info,tidy_data_section_2.csv
2.3 Record type of property,2,3,37,type_of_property,tidy_data_section_2.csv
2.8 Record number of households/families that live on the plot,2,8,52,no_of_hhs,tidy_data_section_2.csv
2.9 How many people live on this plot?,2,9,53,no_of_ppl,tidy_data_section_2.csv
2.10 Record number of toilets on the plot\nVentilated improved latrine (VIP),2,10,55,type_of_toilet,tidy_data_section_2.csv
2.10 Record number of toilets on the plot\nUrine-diversion latrine (Ecosan),2,10,56,type_of_toilet,tidy_data_section_2.csv
2.10 Record number of toilets on the plot\nWaterborne/flush toilet inside the house,2,10,57,type_of_toilet,tidy_data_section_2.csv


## Detailed data
### General background info on respondents and types of toilet
 - note this is not aggregated by plot - what is the unique id?

In [125]:
data1_orig = pd.read_csv(data_dir + 'tidy_data_questionnaire_section_2.csv')
pandas_profiling.ProfileReport(data1_orig)

Number of variables,17
Number of observations,18879
Total Missing (%),25.2%
Total size in memory,2.4 MiB
Average record size in memory,136.0 B
Numeric,7
Categorical,9
Date,0
Text (Unique),0
Rejected,1
Distinct count,51


In [126]:
data1_orig.id.unique().shape, data1_orig.shape

((16065,), (18879, 17))

Find the unique index - is it id and respondent?
It looks like type of toilet might also have a new row

In [127]:
for key, grp in data1_orig.groupby(['id', 'respondent', 'type_of_toilet']):
    if grp.shape[0] > 1:
        print(grp[grp.columns.difference(['id', 'respondent'])].drop_duplicates().shape[0])

We can see that within plot id and type of toilet that there can be multiple respondents but they always give the same responses

In [128]:
for key, grp in data1_orig.groupby(['id','type_of_toilet']):
    if grp.shape[0] > 1:
        dups=grp[grp.columns.difference(['id','type_of_toilet', 'respondent', 'plot_months'])].drop_duplicates().shape[0]
        if dups > 1:
            print(grp)

In [129]:
data1_orig.groupby(['id', 'respondent','type_of_toilet']).size().max()

1

In [130]:
results = data1_orig.groupby('id')[['respondent', 'type_of_toilet']].agg(lambda x:len(set(x)))
results.columns= ['n_respondents', 'n_toilet_types']

In [131]:
data1_orig = data1_orig.set_index('id').join(results)
data1_orig['n_respondents'].value_counts()

1    18800
2       76
3        3
Name: n_respondents, dtype: int64

In [133]:
data1_orig['n_toilet_types'].value_counts()

1    13380
2     5302
3      189
4        8
Name: n_toilet_types, dtype: int64

In [137]:
data1 = data1_orig.reset_index()

In [138]:
data1['respondent_ttype_id'] = data1.groupby('id')['respondent'].transform(lambda x:range(len(x)))

In [139]:
data1['respondent_ttype_id'].value_counts()

0    16065
1     2728
2       75
3       11
Name: respondent_ttype_id, dtype: int64

In [141]:
data1.groupby(['id', 'respondent_ttype_id']).size().max()

1

In [142]:
data1.set_index(['id', 'respondent_ttype_id'], drop=False, inplace=True)

In [143]:
data1.index.unique().shape, data1.shape

((18879,), (18879, 20))

### More specific info on toilet use - unique by toilet user and plot ID?

In [12]:
data3 = pd.read_csv(data_dir + 'tidy_data_questionnaire_section_3.csv')
pandas_profiling.ProfileReport(data3)

Number of variables,4
Number of observations,55467
Total Missing (%),3.6%
Total size in memory,1.7 MiB
Average record size in memory,32.0 B
Numeric,1
Categorical,3
Date,0
Text (Unique),0
Rejected,0
Distinct count,16065


It looks like waste disposal should be in data4, on the toilet type

In [18]:
temp = data3[['id', 'solid_waste_disp']].drop_duplicates()
temp.shape

(16065, 2)

In [22]:
data3.drop('solid_waste_disp', 1, inplace=True)

Unique index check

In [23]:
data3.set_index(['id', 'toilet_user'], inplace=True)

In [26]:
data3.index.unique().shape, data3.shape

((55467,), (55467, 1))

### Data on toilets that is aggregated by plot

In [17]:
data4 = pd.read_csv(data_dir + 'tidy_data_questionnaire_section_4.csv')
pandas_profiling.ProfileReport(data4)

Number of variables,13
Number of observations,16065
Total Missing (%),60.1%
Total size in memory,1.6 MiB
Average record size in memory,104.0 B
Numeric,5
Categorical,7
Date,0
Text (Unique),1
Rejected,0
Distinct count,193


Set the index

In [19]:
data4 = data4.set_index('id')

In [20]:
data4 = data4.join(temp.set_index('id'))

In [28]:
data4.index.unique().shape, data4.shape

((16065,), (16065, 13))

In [21]:
data4.head()

,age_toilet_1,age_toilet_2,age_toilet_3,when_toilet_full,ever_emptied,when_last_emptied,know_emptying_service,who_emptied,how_often_emptied_mo,how_often_emptied_other,when_next_full_mo,why_never_emptied,solid_waste_disp
id,,,,,,,,,,,,,
id_00001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id_00002,36.0,NaN,NaN,empty_and_reuse,yes,between_five_and_six_months,friend_familiy_neighbor,other,NaN,NaN,6.0,NaN,bin_collected_CBEs
id_00003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bin_self_disposed
id_00004,24.0,NaN,NaN,bury_and_build_new,no,NaN,NaN,NaN,NaN,NaN,7.0,never_been_full,rubbish_pit
id_00005,60.0,NaN,NaN,empty_and_reuse,no,NaN,NaN,NaN,NaN,NaN,NaN,never_been_full,rubbish_pit


### Toilets

In [98]:
data5 = pd.read_csv(data_dir + 'tidy_data_questionnaire_section_5.csv')
pandas_profiling.ProfileReport(data5)

Number of variables,14
Number of observations,16065
Total Missing (%),33.7%
Total size in memory,1.7 MiB
Average record size in memory,112.0 B
Numeric,0
Categorical,13
Date,0
Text (Unique),1
Rejected,0
Distinct count,5


# Questions

### (1) How consistent are responses across different people from the same plot

In [38]:
data1.columns

Index(['respondent', 'sex', 'plot_months', 'type_of_property', 'no_of_hhs',
       'no_of_ppl', 's2q9a54', 'type_of_toilet', 'no_of_toilets',
       'add_toilets', 'water_source', 'water_source_other', 'latitude',
       'longitude', 'accuracy', 'altitude'],
      dtype='object')

In [84]:
data1_dups = data1[data1.n_responders>1]

In [94]:
for f in ['no_of_hhs',  'no_of_ppl', 'no_of_toilets']:
    temp = data1_dups.groupby('id')[f].apply(lambda x:list(x))
    temprange = temp.map(lambda x:max(x) - min(x))
    print(f, temprange.max())

/usr/local/anaconda3/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: 'id' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  


no_of_hhs 0.0
no_of_ppl 0.0
no_of_toilets 10.0


There is variance in the no of toilets

In [97]:
data1[data1.id.isin(temprange[temprange==10].index)]

id respondent     sex  plot_months  \
id       respondent_id                                             
id_04493 0              id_04493   landlord  female        240.0   
         1              id_04493   landlord  female        240.0   
id_11131 0              id_11131   landlord  female         60.0   
         1              id_11131   landlord  female         60.0   

                       type_of_property  no_of_hhs  no_of_ppl s2q9a54  \
id       respondent_id                                                  
id_04493 0                  residential        5.0        NaN    True   
         1                  residential        5.0        NaN    True   
id_11131 0                  residential        1.0        7.0     NaN   
         1                  residential        1.0        7.0     NaN   

                            type_of_toilet  no_of_toilets add_toilets  \
id       respondent_id                                                  
id_04493 0               pit_latrine_lined            1.0       yes_1   
         1                  disused_buried           11.0       yes_1   
id_11131 0               pour_flush_inside           11.0       yes_2   
         1              pour_flush_outside            1.0       yes_2   

                                 water_source water_source_other   latitude  \
id       respondent_id                                                        
id_04493 0              individual_connection                NaN -15.441575   
         1              individual_connection                NaN -15.441575   
id_11131 0              individual_connection                NaN -15.443655   
         1              individual_connection                NaN -15.443655   

                        longitude  accuracy  altitude  n_responders  \
id       respondent_id                                                
id_04493 0              28.240803       3.9    1280.0             2   
         1              28.240803       3.9    1280.0             2   
id_11131 0              28.236420       3.9    1263.8             2   
         1              28.236420       3.9    1263.8             2   

                        respondent_id  
id       respondent_id                 
id_04493 0                          0  
         1                          1  
id_11131 0                          0  
         1                          1

Ah... the data set is also split based on type of toilet

### (2) What is the average number of toilets per plot

In [ ]:
data1

### Average number of users per toilet